In [1]:
import pandas as pd
import numpy as np
import pickle as pk
import os
import glob
import zlib

from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold


In [2]:
## Data input

# load graph
with open('dataset_nodes_pdgs_labels.p', 'rb') as fd:
    graph, pdgs, labels = pk.load(fd)

    
labels=pd.Series(labels)
nodes = pd.read_pickle("./features.pkl")



In [3]:
## Features data

print(nodes.head())

   feature1  feature2  feature3   feature4  feature5  feature6  feature7  \
0      17.0      53.0     128.0  12.028289  0.048538  0.249053    1270.0   
1      18.0      54.0     128.0  32.379967 -0.017928  2.538584    1329.0   
2      19.0      54.0     128.0  39.963924 -0.009562  2.760196    1398.0   
3      19.0      54.0     128.0  29.785406 -0.011091  2.890076    1408.0   
4      20.0      54.0     128.0  69.445038 -0.019203  3.075611    1486.0   

   feature8  feature9  
0    4378.0   12097.0  
1    4439.0   12077.0  
2    4432.0   12070.0  
3    4442.0   12080.0  
4    4402.0   12040.0  


In [4]:
## Features:

# Feature1  neighbour_3
# Feature2  neighbour_10
# Feature3  neighbour_30
# Feature4  charge_map
# Feature5  dot_product
# Feature6  angle
# Feature7  charge_distance_3
# Feature8  charge_distance_10
# Feature9  charge_distance_30

# Rename and describe the data frame
nodes.columns = ["neighbour_3", "neighbour_10", "neighbour_30", "charge_map", "dot_product", "angle", "charge_distance_3", "charge_distance_10", "charge_distance_30"]

nodes.describe()

,neighbour_3,neighbour_10,neighbour_30,charge_map,dot_product,angle,charge_distance_3,charge_distance_10,charge_distance_30
count,7.030533e+06,7.030533e+06,7.030533e+06,7.030533e+06,7.030533e+06,7.030533e+06,7.030533e+06,7.030533e+06,7.030533e+06
mean,2.497649e+01,1.113293e+02,3.483956e+02,5.705963e+01,1.804495e-01,1.490602e+00,1.454060e+03,6.613205e+03,2.013232e+04
std,1.076896e+01,5.971821e+01,1.779324e+02,5.215720e+01,1.969706e+01,1.217663e+00,1.110965e+03,4.996266e+03,1.293129e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,5.439834e-03,-2.123361e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.800000e+01,7.200000e+01,2.240000e+02,2.959888e+01,-2.227991e-02,1.451090e-01,8.710000e+02,3.419000e+03,1.068500e+04
50%,2.700000e+01,1.000000e+02,3.080000e+02,4.082213e+01,2.128416e-04,1.484194e+00,1.129000e+03,4.317000e+03,1.452500e+04
75%,3.100000e+01,1.330000e+02,4.510000e+02,6.654980e+01,2.362917e-02,2.857099e+00,1.755000e+03,8.651000e+03,2.895200e+04
max,1.350000e+02,5.320000e+02,1.754000e+03,3.050392e+03,2.849225e+04,3.141593e+00,1.888800e+04,4.613800e+04,1.101830e+05


In [5]:
## Create new labels

# Old labels        
# 0 = track
# 1 = heavy track
# 2 = shower
        
# New labels        
# 0 = track
# 1 = shower

track_number = 0
shower_number = 0
new_labels_list = []
for x in labels:
    if x<2:
        new_labels_list.append(0)
        track_number += 1
    else:
        new_labels_list.append(1)
        shower_number += 1

new_labels = pd.Series(new_labels_list) 
print("Total number of events: ", track_number+shower_number)
print("Total number of tracks: ", track_number)
print("Total number of showers: ", shower_number)
print("Ratio of tracks: ", track_number/(track_number+shower_number))
print("Ratio of showers: ", shower_number/(track_number+shower_number))

Total number of events:  7030533
Total number of tracks:  2315092
Total number of showers:  4715441
Ratio of tracks:  0.32929110780078835
Ratio of showers:  0.6707088921992116


In [6]:
## Split the data

from sklearn.model_selection import train_test_split

X = nodes
#y = labels
y = new_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.6, test_size=None, shuffle=False)


In [7]:
## Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

#scaler = MinMaxScaler()
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:
## Boosted Decision Tree - eXtreme Gradient Boosting

# import XGBoost classifier
from xgboost import XGBClassifier

# import cross_val_score for cross-validation
from sklearn.model_selection import cross_val_score

xgbc = XGBClassifier()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=4, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1) 

xgbc.fit(X_train, y_train)


scores = cross_val_score(xgbc, X_train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())



[16:11:22] WARNING: /workspace/build/pyexternals/xgboost-0.90/src/xgboost/0.90/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
Mean cross-validation score: 0.72


In [9]:
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

## Predicting with confusion matrix
ypred = xgbc.predict(X_test)
cm = confusion_matrix(y_test,ypred) 
print(cm)

K-fold CV average score: 0.73
[[ 472881  446035]
 [ 354903 1538395]]
